<a href="https://colab.research.google.com/github/kumarpriyanshu09/Gemma-3-4B-Fine-tuning-with-Unsloth/blob/main/Finetuning_Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name ="unsloth/gemma-3-4b-it",
    max_seq_length= 2048,
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
)

==((====))==  Unsloth 2025.8.4: Fast Gemma3 patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetuning_vision_layers = False,
    finetuning_language_layers = True,
    finetuning_attention_modules = True,
    finetuning_mlp_modules = True,

    r = 8,
    lora_alpha =8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3047,
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset[96]

{'conversations': [{'content': 'Write the code for the following sorting algorithm, Comb Sort.\n\nComb Sort is an advanced variation of the Bubble Sort algorithm. It works by repeatedly swapping adjacent elements if they are in the wrong order, with a decreasing gap size. The gap size starts with the total number of elements in the list and decreases by a factor of 1.3 with each pass until it reaches 1.\n\nTo implement Comb Sort, follow these steps:\n\n1. Create a function called `combSort` that takes an array of integers as input.\n2. Initialize the gap size to the length of the array.\n3. Create a flag variable called `swapped` and set it to true.\n4. Create a while loop that continues as long as the gap size is greater than 1 or `swapped` is true.\n5. Inside the loop, set `swapped` to false.\n6. Calculate the gap size by dividing the current gap size by 1.3 and rounding it down to the nearest integer.\n7. Create a for loop that iterates from the beginning of the array until the end 

In [ ]:
def formatting_prompts_fuc(examples):
  convos = examples["conversations"]
  texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
  return { "text" : texts , }

dataset = dataset.map(formatting_prompts_fuc, batched = True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset[96]["text"]

'<start_of_turn>user\nWrite the code for the following sorting algorithm, Comb Sort.\n\nComb Sort is an advanced variation of the Bubble Sort algorithm. It works by repeatedly swapping adjacent elements if they are in the wrong order, with a decreasing gap size. The gap size starts with the total number of elements in the list and decreases by a factor of 1.3 with each pass until it reaches 1.\n\nTo implement Comb Sort, follow these steps:\n\n1. Create a function called `combSort` that takes an array of integers as input.\n2. Initialize the gap size to the length of the array.\n3. Create a flag variable called `swapped` and set it to true.\n4. Create a while loop that continues as long as the gap size is greater than 1 or `swapped` is true.\n5. Inside the loop, set `swapped` to false.\n6. Calculate the gap size by dividing the current gap size by 1.3 and rounding it down to the nearest integer.\n7. Create a for loop that iterates from the beginning of the array until the end minus the 

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer =  SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #num_train_epochs = 1,
        max_steps = 32,
        learning_rate = 2e-4,
        logging_steps=1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    )
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import  train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part= "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[96]["input_ids"])

'<bos><start_of_turn>user\nWrite the code for the following sorting algorithm, Comb Sort.\n\nComb Sort is an advanced variation of the Bubble Sort algorithm. It works by repeatedly swapping adjacent elements if they are in the wrong order, with a decreasing gap size. The gap size starts with the total number of elements in the list and decreases by a factor of 1.3 with each pass until it reaches 1.\n\nTo implement Comb Sort, follow these steps:\n\n1. Create a function called `combSort` that takes an array of integers as input.\n2. Initialize the gap size to the length of the array.\n3. Create a flag variable called `swapped` and set it to true.\n4. Create a while loop that continues as long as the gap size is greater than 1 or `swapped` is true.\n5. Inside the loop, set `swapped` to false.\n6. Calculate the gap size by dividing the current gap size by 1.3 and rounding it down to the nearest integer.\n7. Create a for loop that iterates from the beginning of the array until the end minus

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[96]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                                                                                                                                                                                                                                                         Here is the code for the Comb Sort algorithm:\n\n```python\ndef combSort(arr):\n    gap = len(arr)\n    swapped = True\n\n    while gap > 1 or swapped:\n        gap = int(gap / 1.3)\n        if gap < 1:\n            gap = 1\n        swapped = False\n\n        for i in range(len(arr) - gap):\n            if arr[i] > arr[i + gap]:\n                arr[i], arr[i + gap] = arr[i + gap], arr[i]\n                swapped = True\n\n    return arr\n\n# Test the implementation\narr = [5, 8, 2, 1, 9, 3]\nsorted_arr = combSort(arr)\nprint(sorted_arr)\n```\n\nThis code defines the `combSort` funct

In [ ]:
gpu_stats =  torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max Memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max Memory = 14.741 GB.
7.395 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 32
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,248,896 of 4,319,328,368 (0.45% trained)


Step,Training Loss
1,0.911400
2,1.040500
3,1.251100
4,1.045700
5,0.833200
6,1.118200
7,0.789300
8,1.128500
9,1.097000
10,0.976500


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

416.0103 seconds used for training.
6.93 minutes used for training.
Peak reserved memory = 7.395 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 50.166 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\nThe sequence is the Fibonacci sequence, which means that each number is the sum of the two previous numbers in the sequence.\n\n= 8 + 12 = 20\n\nPossible numbers: 1, 1, 2, 3, 5, 8, 15, 1']

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type": "text", "text": "why is the sky blue?"}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt =True,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = 'pt').to("cuda"),
    max_new_tokens = 64,
    temperature = 1.0,
    top_p = 0.95,
    top_k = 64,
    streamer = TextStreamer(tokenizer, skip_promot = True),
)

<bos><start_of_turn>user
why is the sky blue?<end_of_turn>
<start_of_turn>model
The sky is blue because of a phenomenon called Rayleigh scattering, which is where air molecules scatter sunlight differently depending on the wavelength of light. The higher frequencies of red and orange light are more likely to be scattered, while the lower frequencies of blue and violet light are more likely to pass straight through.

The atmosphere is the


In [ ]:
model.save_pretrained("gemma-3-lora")
tokenizer.save_pretrained("gemma-3-lora")


In [ ]:
model.save_pretrained_merged("gemma-3-finetune", tokenizer)

In [ ]:
model.save_pretrained_gguf(
    "gemma-3-finetune",
    quantization_type="Q8_0"
)

In [ ]:
if True:
  from unsloth import FastModel
  model,tokenzizer = FastModel.from_pretrained(
      model_name = "path/to/saved/model",
      load_in_4bit = True,
  )

  messages = [{
      "role" : "user",
      "content": [{"type": "text", "text": "why is the Gemma-3?"}]
  }]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = 'pt').to("cuda"),
    max_new_tokens = 64,
    temperature = 1.0,
    top_p = 0.95,
    top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

==((====))==  Unsloth 2025.8.4: Fast Gemma3 patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Gemma-3 is a family of large language models (LLMs) released by the AI startup, Google. It is based on the Llama foundation model, where is currently available in sizes ranging from 20 gigabytes to 80 gigabytes. 

Gemma-3 models are intended for both
